## Assignment 5 : Document Classification

#### Dilip Ganesan, Raghu Ramnath and  Ahmed

It can be useful to be able to classify new "test" documents using already classified "training" documents.  A common example is using a corpus of labeled spam and ham (non-spam) e-mails to predict whether or not a new document is spam.  Here is one example of such data:  http://archive.ics.uci.edu/ml/datasets/Spambase

For this project, you can either use the above dataset to predict the class of new documents (either withheld from the training dataset or from another source such as your own spam folder).

For more adventurous students, you are welcome (encouraged!) to come up a different set of documents (including scraped web pages!?) that have already been classified (e.g. tagged), then analyze these documents to predict how new documents should be classified.

This assignment is due end of day on Monday, 11/12.  You may work in a small team if you want

### Spam Document Classifier Using Random Forest Classifier.

In [1]:
import nltk

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [2]:
# Loading of SPAMDATA SET
spamdataset = pd.read_csv("https://raw.githubusercontent.com/dilipganesan/DATA620_WebAnalytics_GroupProject/master/Assignment%205/spamfiles/spamdata.csv")

In [3]:
spamdataset.describe()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,...,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,0.104553,0.213015,0.280656,0.065425,0.312223,0.095901,0.114208,0.105295,0.090067,0.239413,...,0.038575,0.139030,0.016976,0.269071,0.075811,0.044238,5.191515,52.172789,283.289285,0.394045
std,0.305358,1.290575,0.504143,1.395151,0.672513,0.273824,0.391441,0.401071,0.278616,0.644755,...,0.243471,0.270355,0.109394,0.815672,0.245882,0.429342,31.729449,194.891310,606.347851,0.488698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.065000,0.000000,0.000000,0.000000,0.000000,2.276000,15.000000,95.000000,0.000000
75%,0.000000,0.000000,0.420000,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.160000,...,0.000000,0.188000,0.000000,0.315000,0.052000,0.000000,3.706000,43.000000,266.000000,1.000000
max,4.540000,14.280000,5.100000,42.810000,10.000000,5.880000,7.270000,11.110000,5.260000,18.180000,...,4.385000,9.752000,4.081000,32.478000,6.003000,19.829000,1102.500000,9989.000000,15841.000000,1.000000


In [4]:
spamdataset.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [29]:
spamdataset.dtypes

word_freq_make                float64
word_freq_address             float64
word_freq_all                 float64
word_freq_3d                  float64
word_freq_our                 float64
word_freq_over                float64
word_freq_remove              float64
word_freq_internet            float64
word_freq_order               float64
word_freq_mail                float64
word_freq_receive             float64
word_freq_will                float64
word_freq_people              float64
word_freq_report              float64
word_freq_addresses           float64
word_freq_free                float64
word_freq_business            float64
word_freq_email               float64
word_freq_you                 float64
word_freq_credit              float64
word_freq_your                float64
word_freq_font                float64
word_freq_000                 float64
word_freq_money               float64
word_freq_hp                  float64
word_freq_hpl                 float64
word_freq_ge

Spam is the response variable. We would like to see how many of these are Spam and how many are Ham.

In [5]:
spamcount = len(spamdataset[spamdataset.spam==1])
hamcount = len(spamdataset[spamdataset.spam==0])

We will print and see the spam and ham counts. 

In [6]:
print(spamcount)
print(hamcount)

1813
2788


From the above we can see that the Spam email are 1813.

This is a classification problem. We will be using Random Forest and SVM Classifier to classify spam vs ham.
Before starting on the classification, we have to split the dataset in to training datatset and testing dataset.

We know the last column spam is the response variable Y and rest of the variables will be split in to X.

In [7]:
X = spamdataset.values[:, 0:57]
Y = spamdataset.values[:, 57]

Next we will use the train_test_split from sklearn package to split the spamdataset into X_train, X_test, Y_train, Y_test.

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 38)


## Spam Document Classifier Using Random Forest

In [10]:
model = RandomForestClassifier(random_state=42) 
model.fit(X_train, Y_train)

# Predict for test data sample
model_prediction_train = model.predict(X_test)

In [11]:
def print_model_output(sample_test, prediction, score=None):
    con_mat = metrics.confusion_matrix(sample_test, prediction)
    classreport = metrics.classification_report(sample_test, prediction)
    try:
        print('Random Forrest Classifier Model:')
        print(prediction)
        print('\nConfusion Matrix:')
        print(con_mat)
        print('\nClassification Rport:')
        print(classreport)
    except: 
        raise
 

In [12]:
score = model.score(X_test, Y_test)
print_model_output(Y_test, model_prediction_train, score=score)

Random Forrest Classifier Model:
[0. 0. 0. ... 0. 0. 1.]

Confusion Matrix:
[[824  31]
 [ 45 481]]

Classification Rport:
             precision    recall  f1-score   support

        0.0       0.95      0.96      0.96       855
        1.0       0.94      0.91      0.93       526

avg / total       0.94      0.94      0.94      1381



In [13]:
accuracy_score(Y_test,model_prediction_train)

0.944967414916727

In [22]:
# Random forest feature importance
# display the relative importance of each attribute
importances = model.feature_importances_
#Sort it
print ("Sorted Feature Importance:")
sorted_feature_importance = sorted(zip(importances, list(X_train)), reverse=True)
print(sorted_feature_importance[0])

Sorted Feature Importance:
(0.1357694675873437, array([ 0.   ,  1.08 ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  1.08 ,  0.   ,  3.26 ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  5.43 ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  1.08 ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.169,  0.   ,  0.   ,  2.052, 21.   ,
       39.   ]))


#### Random Forest Classifier has an accuracy of 94%

## Spam Document Classifier Using Support Vector Machine.

Similar to Random Forest Classifier, we will use the SVM to see the accuracy and classification report.

In [14]:
svm = svm.SVC(random_state=1)
svm.fit(X_train, Y_train)

svm_prediction_train = svm.predict(X_test)


In [15]:
def print_svm_output(sample_test, prediction, score=None):
    con_mat = metrics.confusion_matrix(sample_test, prediction)
    classreport = metrics.classification_report(sample_test, prediction)
    try:
        print('Support Vector Machine Classifier:')
        print(prediction)
        print('\nConfusion Matrix:')
        print(con_mat)
        print('\nClassification Rport:')
        print(classreport)
    except: 
        raise

In [16]:
svmscore = svm.score(X_test, Y_test)
print_svm_output(Y_test,svm_prediction_train, score=score)

Support Vector Machine Classifier:
[0. 0. 0. ... 0. 0. 1.]

Confusion Matrix:
[[735 120]
 [105 421]]

Classification Rport:
             precision    recall  f1-score   support

        0.0       0.88      0.86      0.87       855
        1.0       0.78      0.80      0.79       526

avg / total       0.84      0.84      0.84      1381



In [51]:
accuracy_score(Y_test,svm_prediction_train)

0.83707458363504705

## Spam Document Classifier Using Decision Tree.

In [26]:
from sklearn import tree

modelt = tree.DecisionTreeClassifier()

In [30]:
#Let’s take a look at our model’s attributes
modelt.fit(X_train, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [31]:
#Then we score the predicted output from model on our test data against our ground truth test data.
y_predict = modelt.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_score(Y_test, y_predict)

0.9051412020275162

We see an accuracy score of 90%, which is significantly better than SVM but less than Random Forest classification.
Let’s also take a look at our confusion matrix:

In [33]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(
    confusion_matrix(Y_test, y_predict),
    columns=['Predicted Negative', 'Predicted Positive'],
    index=['True Negative', 'True Positive']
)

,Predicted Negative,Predicted Positive
True Negative,779,76
True Positive,55,471


## Conclusion:

Spam is becoming a serious threat for regular users of emails, businesses and corporate firms. In this assignment, we have applied three different algorithms(Random Forest, SVM and decision tree) on the set of data  which consist of spam and ham mails collected from our sources. Result analysis on test data shows that Ramdom Forest outperforms SVM and decision tree classifiers when it comes to accuracy.  

Random Forest --> 94%

Support Vector Machine --> 83%

Decision Tree--> 90%